In [2]:
USE master;
GO
CREATE SERVER AUDIT Custom_Audit 
    TO FILE (FILEPATH='D:\Data\')
    WITH (QUEUE_DELAY = 5000);
GO
ALTER SERVER AUDIT Custom_Audit WITH (STATE = ON);
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.004

In [3]:
CREATE SERVER AUDIT SPECIFICATION UserDefinedEvents
FOR SERVER AUDIT Custom_Audit
ADD (USER_DEFINED_AUDIT_GROUP)
WITH (STATE = ON);
GO

Commands completed successfully.

Total execution time: 00:00:00.001

In [4]:
EXEC sp_audit_write @user_defined_event_id = 999, 
                    @succeeded = 1,
                    @user_defined_information = N'Example call to sp_audit_write';

Commands completed successfully.

Total execution time: 00:00:00

In [6]:

SELECT user_defined_event_id, succeeded, user_defined_information
FROM sys.fn_get_audit_file ('D:\Data\Custom_Audit*',default,default)
WHERE user_defined_event_id = 999;

(1 row affected)

Total execution time: 00:00:00.030

user_defined_event_id,succeeded,user_defined_information
999,1,Example call to sp_audit_write


In [7]:
USE salesapp1;
GO

CREATE PROC usp_OrderDetailDiscount
	@orderid int,
	@productid int,
	@discount numeric(4,3)
AS
	SET NOCOUNT ON

	IF @discount > 0.3
	BEGIN
		DECLARE @msg nvarchar(4000) = 
		  CONCAT('Order=',@orderid,':Product=',@productid,
		         ':discount=', @discount)

		
		EXEC sp_audit_write @user_defined_event_id = 998, 
				            @succeeded = 1,
						    @user_defined_information = @msg;
	END

	UPDATE Sales.OrderDetails
	SET discount = @discount
	WHERE orderid = @orderid
	AND productid = @productid
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.003

In [8]:

EXEC dbo.usp_OrderDetailDiscount @orderid = 10248,@productid =	11, @discount = 0.05
EXEC dbo.usp_OrderDetailDiscount @orderid = 10248,@productid =	42, @discount = 0.45

Commands completed successfully.

Total execution time: 00:00:00.395

In [10]:
SELECT user_defined_event_id, succeeded, user_defined_information
FROM sys.fn_get_audit_file ('D:\Data\Custom_Audit*',default,default)
WHERE user_defined_event_id = 998;

(1 row affected)

Total execution time: 00:00:00.030

user_defined_event_id,succeeded,user_defined_information
998,1,Order=10248:Product=42:discount=0.450


In [0]:
USE master;
GO
ALTER SERVER AUDIT Custom_Audit WITH (STATE = OFF);
DROP SERVER AUDIT Custom_Audit;
GO

ALTER SERVER AUDIT SPECIFICATION UserDefinedEvents WITH (STATE = OFF);
DROP SERVER AUDIT SPECIFICATION UserDefinedEvents
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.007